# Info

This script analyzes TPD data. 

The following external packages must be installed:
- lmfit
- igor
- plotly
- xarray

To install standard external packages open a terminal (Anaconda Prompt in Windows or another terminal if using Linux/Mac) and type:

<p style="margin-left: 40px">pip install [package name]</p>

To use widgets wtih Jupyter Lab, open a terminal window and type:

<p style="margin-left: 40px">conda install -c conda-forge nodejs</p>
<p style="margin-left: 40px">jupyter labextension install @jupyter-widgets/jupyterlab-manager</p>

To show plotly figures correctly, open a terminal window and type:
<p style="margin-left: 40px">jupyter labextension install @jupyterlab/plotly-extension</p>

-----------------------
Created by Jerry LaRue, larue@chapman.edu, 12/2018

Last modified by Jerry LaRue, larue@chapman.edu, 07/2020

# Select file

In [1]:
import numpy as np
import igor.igorpy as igor
import os
from os import listdir
from os.path import isfile, join
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import clear_output
import math
import struct
from importlib import reload
import plotly.express as px
from pandas import DataFrame as df

sys.path.append(os.getcwd() + '/Tools/')
import TDSTools
reload(TDSTools)

dt = TDSTools.DataTools()

ParameterFolder = os.getcwd()+'/TDS'

##### Widgets #####

ParameterFile = widgets.Dropdown(
    options=dt.FileList(ParameterFolder,['.yaml']),
    description='Select File',
    layout=Layout(width='70%'),
    style = {'description_width': '150px'},
    disabled=False,
)

display(ParameterFile)

##### Functions #####

def FileList() :
    return [f for f in listdir(FolderPath.value) if isfile(join(FolderPath.value, f))]

Dropdown(description='Select File', layout=Layout(width='70%'), options=('TPD181019_01 - CH3OH_O2_Ni', 'TPD181…

In [2]:
##### Load Data #####

tds = TDSTools.TDS((ParameterFolder,ParameterFile.value))
Data = tds.Data
Data = dt.TrimData(Data,115,2000)
Masses = tds.Parameters['Masses']

##### Functions #####

def SimulateTrace_Clicked(b) :
    with out :
        clear_output(True)
        
        try :
            tds.Parameters['HeatingRate'][0]
        except : 
            Rate = 60                 # K/min
        else :
            Rate = tds.Parameters['HeatingRate'][0]
        
        kB = 8.617e-5                 # eV/K
        yOffset = Offset.value
        Scaling = ScalingFactor.value
        
        IntRate1 = 0
        Ni1 = InitialCoverage1.value
        Ea1 = DesorptionBarrier1.value
        nu1 = Prefactor1.value
        n1 = ReactionOrder1.value
        
        IntRate2 = 0
        Ni2 = InitialCoverage2.value
        Ea2 = DesorptionBarrier2.value
        nu2 = Prefactor2.value
        n2 = ReactionOrder2.value

        Size = len(Data)
        Time = np.zeros((Size))
        Trace1 = np.zeros((Size))
        Coverage1 = np.zeros((Size))
        Trace2 = np.zeros((Size))
        Coverage2 = np.zeros((Size))

        for idx, Temperature in enumerate(Data['Temperature (K)']) :
            Trace1[idx] = nu1*(Ni1 - IntRate1)**n1 * np.exp(-Ea1/(kB*Temperature))
            Trace2[idx] = nu2*(Ni2 - IntRate2)**n2 * np.exp(-Ea2/(kB*Temperature))
            if idx != 0 :
                Time[idx] = PreviousTime + (Temperature - PreviousTemperature) / Rate
                IntRate1 += Trace1[idx] * (Temperature - PreviousTemperature)
                IntRate2 += Trace2[idx] * (Temperature - PreviousTemperature)
            Coverage1[idx] = Ni1 - IntRate1
            Coverage2[idx] = Ni2 - IntRate2
            if IntRate1 >= Ni1 :
                IntRate1 = Ni1
                Coverage1[idx] = 0
            if IntRate2 >= Ni2 :
                IntRate2 = Ni2
                Coverage2[idx] = 0
            if Coverage1[idx] < 0 or Coverage1[idx] > Ni1 :
                Coverage1[idx] = 0
                Trace1[idx] = 0
            if Coverage2[idx] < 0 or Coverage2[idx] > Ni2 :
                Coverage2[idx] = 0
                Trace2[idx] = 0
            PreviousTime = Time[idx]
            PreviousTemperature = Temperature
        
        Trace = Trace1 + Trace2
        Data['Sim'] = Trace*Scaling + yOffset
        fig = px.line(Data,x='Temperature (K)',y=Data.columns)
        fig.update_layout(yaxis_title='Intensity (au)',title_text=tds.Parameters['Description']+' with simulated trace',legend_title='',height=500)
        fig.show()

        CoverageSim = np.array((Data['Temperature (K)'].values,Coverage1+Coverage2))
        CoverageSim = df(np.transpose(CoverageSim),columns=('Temperature (K)','Coverage'))
        fig = px.line(CoverageSim,x='Temperature (K)',y=CoverageSim.columns)
        fig.update_layout(yaxis_title='Coverage',showlegend=False,height=100)
        fig.show()

##### Widgets #####

Offset = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000000,
    step=0.1,
    description='Offset',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

ScalingFactor = widgets.BoundedFloatText(
    value=10000000,
    min=0,
    max=1000000000,
    step=0.1,
    description='Scaling Factor',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

InitialCoverage1 = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=1,
    step=0.01,
    description='Initial Coverage',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

DesorptionBarrier1 = widgets.BoundedFloatText(
    value=0.7,
    min=0,
    max=10,
    step=0.01,
    description='Desorption Barrier (eV)',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

Prefactor1 = widgets.BoundedFloatText(
    value=1e10,
    min=0,
    max=1e30,
    step=1,
    description='Prefactor (Hz)',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

ReactionOrder1 = widgets.BoundedFloatText(
    value=1,
    min=1,
    max=2,
    step=1,
    description='Reaction Order',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

InitialCoverage2 = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1,
    step=0.01,
    description='Initial Coverage',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

DesorptionBarrier2 = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=10,
    step=0.01,
    description='Desorption Barrier (eV)',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

Prefactor2 = widgets.BoundedFloatText(
    value=1e10,
    min=0,
    max=1e30,
    step=1,
    description='Prefactor (Hz)',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

ReactionOrder2 = widgets.BoundedFloatText(
    value=1,
    min=1,
    max=2,
    step=1,
    description='Reaction Order',
    layout=Layout(width='25%'),
    style = {'description_width': '150px'},
    disabled=False
)

SimulateTrace = widgets.Button(description="Simulate Trace")
SimulateTrace.on_click(SimulateTrace_Clicked)

display(widgets.Box([Offset,ScalingFactor]))
display(widgets.Box([InitialCoverage1,DesorptionBarrier1,Prefactor1,ReactionOrder1]))
display(widgets.Box([InitialCoverage2,DesorptionBarrier2,Prefactor2,ReactionOrder2]))

display(SimulateTrace)

out = widgets.Output()

##### Plot Data #####

with out :
    fig = px.line(Data,x='Temperature (K)',y=Data.columns)
    fig.update_layout(yaxis_title='Intensity (au)',title_text=tds.Parameters['Description'],legend_title='',height=500)
    fig.show()

out

Box(children=(BoundedFloatText(value=0.0, description='Offset', layout=Layout(width='25%'), max=1000000.0, ste…

Box(children=(BoundedFloatText(value=1.0, description='Initial Coverage', layout=Layout(width='25%'), max=1.0,…

Box(children=(BoundedFloatText(value=0.0, description='Initial Coverage', layout=Layout(width='25%'), max=1.0,…

Button(description='Simulate Trace', style=ButtonStyle())

Output()

In [10]:

#Make indexes nice to work with
def indexTrim(Data) :
    boo = np.zeros(len(Data), int)
    for a, b in enumerate(boo) :
        boo[a] = a
    Data = Data.set_index(boo)
    return Data

#picking a range
def DataRange(d, lower, upper) :
    td = d.iloc[lower:upper]
    return td

#find mean of points in a dataframe for a given mass
def Baseline(d, mass) :
    base = np.mean(d[mass])
    return base

def integrate(d, base, mass) :
    #get deltaT by just taking the difference between first two temp values

    integrateSum = 0
    first = 0
    
    while first < len(d)-1 :
        deltaT = d['Temperature (K)'].values[1] -d['Temperature (K)'].values[0]
        #print(first)
        second =  first + 1

        y1 = float(d[mass].values[first])
        y2 = float(d[mass].values[second])
        deltaY = y2-y1
  
        rectA = deltaT * (y1-base)
        triA = deltaT * (deltaY)/2

        integrateSum = integrateSum + rectA + triA
        first = first + 1
        
    return integrateSum


d = indexTrim(Data)
baserange = DataRange(d, 50, 100)
base = Baseline(baserange, 'Mass 28')
print('Base is :', base)
print('Base integration check is :', integrate(baserange, base, 'Mass 28'))
print('integrated value is :', integrate(d, base, 'Mass 28'))

Base is : 12308.0
Base integration check is : -784.9920000000129
integrated value is : 2863565.6959999837


In [9]:
(np.sum(Data['Mass 28']-12308.0))*(Data['Temperature (K)'].values[1] -Data['Temperature (K)'].values[0])

2868237.567999986